In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install rdkit
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 72.6 MB/s eta 0:00:00


In [3]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import average_precision_score
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import duckdb

2024-06-02 14:37:11.808286: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-02 14:37:11.808427: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-02 14:37:11.980775: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

In [5]:
con = duckdb.connect()

# 각 단백질에 대해 binds=0,1 데이터를 각각 12000개씩 불러오기
df_brd4_0 = con.query(f"""
    SELECT * 
    FROM parquet_scan('{train_path}') 
    WHERE binds = 0 AND protein_name = 'BRD4'
    ORDER BY random()
    LIMIT 20000
""").df()

df_brd4_1 = con.query(f"""
    SELECT * 
    FROM parquet_scan('{train_path}') 
    WHERE binds = 1 AND protein_name = 'BRD4'
    ORDER BY random()
    LIMIT 20000
""").df()

df_hsa_0 = con.query(f"""
    SELECT * 
    FROM parquet_scan('{train_path}') 
    WHERE binds = 0 AND protein_name = 'HSA'
    ORDER BY random()
    LIMIT 20000
""").df()

df_hsa_1 = con.query(f"""
    SELECT * 
    FROM parquet_scan('{train_path}') 
    WHERE binds = 1 AND protein_name = 'HSA'
    ORDER BY random()
    LIMIT 20000
""").df()

df_seh_0 = con.query(f"""
    SELECT * 
    FROM parquet_scan('{train_path}') 
    WHERE binds = 0 AND protein_name = 'sEH'
    ORDER BY random()
    LIMIT 20000
""").df()

df_seh_1 = con.query(f"""
    SELECT * 
    FROM parquet_scan('{train_path}') 
    WHERE binds = 1 AND protein_name = 'sEH'
    ORDER BY random()
    LIMIT 20000
""").df()

# 데이터프레임 결합
df = pd.concat([df_brd4_0, df_brd4_1, df_hsa_0, df_hsa_1, df_seh_0, df_seh_1], axis=0).reset_index(drop=True)
con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [6]:
# ECFP 생성 함수
def generate_ecfp(molecule, radius=2, bits=2048):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

In [7]:
# 분자의 물리화학적 특성을 추출하는 함수
def generate_physchem_features(molecule):
    if molecule is None:
        return [np.nan] * 7
    return [
        Descriptors.MolWt(molecule),                   # 분자량
        Descriptors.MolLogP(molecule),                 # 로그 P
        Descriptors.NumHDonors(molecule),              # 수소 결합 공여자 수
        Descriptors.NumHAcceptors(molecule),           # 수소 결합 수용체 수
        Descriptors.TPSA(molecule),                    # TPSA (극성 표면적)
        Descriptors.NumRotatableBonds(molecule),       # 회전 가능한 결합 수
        Descriptors.RingCount(molecule)                # 고리의 수
    ]

In [8]:
# 데이터 전처리
df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles)
df['physchem'] = df['molecule'].apply(generate_physchem_features)
df['ecfp'] = df['molecule'].apply(generate_ecfp)

In [9]:
# 물리화학적 특성을 정규화
physchem_features = np.array(df['physchem'].tolist())
scaler = StandardScaler()
physchem_features_scaled = scaler.fit_transform(physchem_features)

In [10]:
# ECFP와 정규화된 물리화학적 특성을 결합
df['features'] = [ecfp + physchem_scaled.tolist() for ecfp, physchem_scaled in zip(df['ecfp'], physchem_features_scaled)]

In [11]:
# 단백질 이름을 원-핫 인코딩
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))

In [12]:
# 최종 입력 데이터 생성
X = np.array([features + list(protein) for features, protein in zip(df['features'].tolist(), protein_onehot.tolist())])
y = np.array(df['binds'].tolist())

In [13]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# 신경망 모델 정의
model = Sequential()
model.add(Dense(1024, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

In [15]:
# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
# 모델 학습
model.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.2, verbose=1)

Epoch 1/100
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 33s 26ms/step - accuracy: 0.7346 - loss: 0.5084 - val_accuracy: 0.8840 - val_loss: 0.2885
Epoch 2/100
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 40s 25ms/step - accuracy: 0.8796 - loss: 0.2969 - val_accuracy: 0.8959 - val_loss: 0.2516
Epoch 3/100
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 34s 28ms/step - accuracy: 0.9043 - loss: 0.2407 - val_accuracy: 0.9056 - val_loss: 0.2309
Epoch 4/100
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 33s 27ms/step - accuracy: 0.9185 - loss: 0.2021 - val_accuracy: 0.9108 - val_loss: 0.2195
Epoch 5/100
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 32s 26ms/step - accuracy: 0.9349 - loss: 0.1661 - val_accuracy: 0.9138 - val_loss: 0.2188
Epoch 6/100
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 30s 25ms/step - accuracy: 0.9477 - loss: 0.1358 - val_accuracy: 0.9170 - val_loss: 0.2237
Epoch 7/100
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 32s 27ms/step - accuracy: 0.9553 - loss: 0.1180 - val_accuracy: 0.9162 - val_loss: 0.2287
Epoch 8/100
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 32s 26ms/step - accuracy: 

In [17]:
# 예측
y_pred_proba = model.predict(X_test).flatten()
map_score = average_precision_score(y_test, y_pred_proba)
print(f"Mean Average Precision (MAP) Score: {map_score}")

750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step
Mean Average Precision (MAP) Score: 0.9663069425099892


In [18]:
# 테스트 데이터 예측 및 저장
test_file = '/kaggle/input/leash-BELKA/test.csv'  
output_file = '/kaggle/working/submission8.csv'  # 출력 파일 경로

In [19]:
import os

for df_test in pd.read_csv(test_file, chunksize=100000):
    df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
    df_test['ecfp'] = df_test['molecule'].apply(generate_ecfp)
    df_test['physchem'] = df_test['molecule'].apply(generate_physchem_features)
    
    # 테스트 데이터의 물리화학적 특성 정규화
    physchem_features_test = np.array(df_test['physchem'].tolist())
    physchem_features_test_scaled = scaler.transform(physchem_features_test)
    
    df_test['features'] = [ecfp + physchem_scaled.tolist() for ecfp, physchem_scaled in zip(df_test['ecfp'], physchem_features_test_scaled)]
    protein_onehot = onehot_encoder.transform(df_test['protein_name'].values.reshape(-1, 1))
    X_test = np.array([features + list(protein) for features, protein in zip(df_test['features'].tolist(), protein_onehot.tolist())])
    
    probabilities = model.predict(X_test).flatten()
    output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities})
    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step
2341/2341 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step
